In [11]:
import os
import sys
import ccxt
import pandas as pd
import collections


file = "C:/Users/Carter Carlson/Documents/Excel references/secret.csv"
api = pd.read_csv(file)

binance = ccxt.binance({
    'options':{'adjustForTimeDifference':True},
    'apiKey': api['apiKey'][0],
    'secret': api['secret'][0]
})

balance = binance.fetchBalance()
coins = ['BTC', 'ETH', 'XRP', 'OMG']

data = pd.DataFrame(columns=['symbol', 'quantity', 'price', 'dollar_value'])
btc_price = float(binance.fetch_ticker('BTC/USDT')['info']['lastPrice'])
total_value = 0

for coin in coins:
    quantity = balance[coin]['total']
    
    if coin == 'BTC':
        price = btc_price
    else:
        btc_ratio = float(binance.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
        price = btc_ratio * btc_price

    dollar_value = quantity * price
    total_value += dollar_value

    data = data.append({'symbol': coin,
                        'quantity': quantity,
                        'price': price,
                        'dollar_value': dollar_value},
                      ignore_index=True)

data['weight'] = list(map(lambda x: x / total_value, data['dollar_value']))

In [13]:
data[data['weight'] > .2]
# list of all weights
# of that, only weights above 30% remain
# get min weight
# figure it out

,symbol,quantity,price,dollar_value,weight
0,BTC,0.227508,8190.730000,1863.456355,0.206339
1,ETH,5.226496,455.273536,2379.485417,0.263479
2,XRP,7722.869000,0.446477,3448.081006,0.381804


In [ ]:
symbol = 'XRP/ETH'
type = 'market'
price = None

diff = abs(xrpWeight - ethWeight)
if diff > .01:
    if xrpWeight > ethWeight:
        side = 'sell'
    else:
        side = 'buy'
    # Have to account for fees
    rate = diff / 2 + diff * .01
    cashAmount = rate * totalValue
    amount = cashAmount / xrpTicker

    #print(binance.create_order(symbol, type, side, amount))

In [ ]:
balance = binance.fetchBalance()
ethBalance = balance['ETH']['free']
xrpBalance = balance['XRP']['free']

ethValue = ethTicker * ethBalance
xrpValue = xrpTicker * xrpBalance

totalValue = ethValue + xrpValue
ethWeight = ethValue / totalValue
xrpWeight = xrpValue / totalValue
print('ETH weight: ' + str(ethWeight) + '\nXRP weight: ' + str(xrpWeight))
print('ETH value: ' + str(ethValue) + '\nXRP value: ' + str(xrpValue))

In [ ]:
# note: dynamically add coins using binance.balance()
# Check if we need more BNB for transaction fees
bnb_quantity = balance['BNB']['total']

if bnb_quantity < 1:
    max_val = 0
    # find coin used to buy more BNB
    for item in data.items():
        if item[1]['dollar_value'] > max_val and item[0] != 'OMG' and item[0] != 'XRP':
            max_val = item[1]['dollar_value']
            coin_for_bnb = item[0]
            # Note: figure how to trade - BNB/BTC and BNB/ETH, but XRP/BNB

In [ ]:
# Old, entertaining(and sometimes confusing?) code
coins = ['BTC', 'ETH', 'XRP', 'OMG']
data = collections.defaultdict(dict)
total_value = 0
---------
for coin in coins:
    # Quantity of coin
    data[coin]['quantity'] = balance[coin]['total']
    btc_price = float(binance.fetch_ticker('BTC/USDT')['info']['lastPrice'])
    # Price of coin (converted from BTC to $)
    if coin == 'BTC':
        data[coin]['price'] = btc_price
    else:
        btc_ratio = float(binance.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
        data[coin]['price'] = btc_ratio * btc_price
    
    # Dollar value 
    data[coin]['dollar_value'] = data[coin]['quantity'] * data[coin]['price']
    total_value += data[coin]['dollar_value']

# Add relative weight of coins
data[coin]['weight'] = [data[coin]['dollar_value'] / total_value for coin in coins]
-----------